In [1]:
import numpy as np
from scipy.signal import convolve2d

actBits = 8
weightBits = 8
outBits = 8
nActs = 10
nWeights = 3

np.random.seed(0)
a = np.random.randint(-2**(actBits-1),2**(actBits-1)-1,size=(nActs,nActs))
w = np.random.randint(-2**(weightBits-1),2**(weightBits-1)-1,size=(nWeights,nWeights))
o = convolve2d(a,w[::-1].T[::-1].T,mode='valid')

256

In [2]:
import numpy as np
from scipy.signal import convolve2d

actBits = 3
weightBits = 3
nActs = 10
nWeights = 3

act_max_value = 2**(actBits-1) - 1
weight_max_value = 2**(weightBits-1) - 1
a = np.full((nActs, nActs), act_max_value)
w = np.full((nWeights, nWeights), weight_max_value)
o = convolve2d(a,w[::-1].T[::-1].T,mode='valid')

Simulation of Quantization per https://github.com/google/gemmlowp/blob/master/doc/quantization.md

In [2]:
# Experiments on low-precision convolution

from quant import *

actBits = 8
weightBits = 8
outBits = 8
nActs = 10
nWeights = 3
nOuts = nActs - nWeights + 1

a = quantized_tensor((nActs,nActs),actBits) 
w = quantized_tensor((nWeights,nWeights),weightBits)
o = scaling_quantized_convolution(a,w,outBits,internalPrecision=16)

o.real_values - convolve_fake_quantized(a,w)
# np.round(o.real_values,3) - np.round(convolve_reals(a,w),3)
# np.allclose(o.real_values,convolve_reals(a,w),rtol=0.001)
# Limit numpy print precision
np.set_printoptions(precision=4)
o.real_values, convolve_fake_quantized(a,w)


(array([[-0.0562,  1.3475, -1.0504, -1.0316,  0.8833,  1.675 , -1.4095,
          0.4754],
        [ 0.1941,  1.3914,  1.0818, -0.6051,  0.7486, -0.4316,  1.6213,
         -1.7036],
        [-2.5406,  0.8668,  1.2382, -1.5457, -2.5349, -0.2429,  0.166 ,
          0.4121],
        [ 1.5829, -2.0628, -0.9616,  3.2697, -0.4504, -0.3762, -0.0659,
          0.2627],
        [ 0.763 ,  2.1768, -0.136 , -1.2791,  1.535 ,  0.2247, -2.1347,
          0.5068],
        [-0.8132, -0.4034, -0.9699, -1.1228, -1.9253, -0.0043,  0.8805,
          0.2682],
        [-0.9916, -0.5115, -0.5937,  0.3508,  1.3887, -0.3928,  1.1233,
         -1.04  ],
        [-0.3788,  1.7169, -0.3493,  1.944 ,  0.7403, -0.6637, -0.1038,
          0.1926]]),
 array([[-0.0562,  1.3475, -1.0504, -1.0316,  0.8833,  1.675 , -1.4095,
          0.4754],
        [ 0.1941,  1.3914,  1.0819, -0.6051,  0.7486, -0.4316,  1.6213,
         -1.7037],
        [-2.5406,  0.8668,  1.2382, -1.5457, -2.5349, -0.2429,  0.166 ,
          0.4121

In [7]:
# Experiments on the fixed point scaling
outBits = 8
out_scale = 2 / (2**outBits - 1)
m = a.scale * w.scale / out_scale
m0, shift = convert_scale_to_shift_and_m0(a.scale * w.scale / out_scale)
print(m,m0 * 2**shift)
print(f'fixed point m0: {convert_to_fixed_point(m0,16)}')
print(shift)
print(m0)

0.00784313725490196 0.007843017578125
fixed point m0: 1000000010000000
-6.0
0.501953125
